In [1]:
import findspark # check if pyspark is installed
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt

In [9]:
import os
current_directory = os.getcwd()
dataset_folder = os.path.join(current_directory, 'datasets')

In [2]:
findspark.find()
findspark.init()

In [3]:
spark = SparkSession \
        .builder \
        .appName("practiseSession") \
        .getOrCreate()

22/02/18 11:59:53 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.188 instead (on interface wlp3s0)
22/02/18 11:59:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/18 11:59:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[You can find more options in the docs](https://spark.apache.org/docs/latest/sql-data-sources-csv.html#data-source-option)

In [13]:
def load_dataframe(filename):
    df = spark.read.format('csv').options(header='true').options(infer_schema='true').load(filename)
    return df

In [121]:
raw_movies_df = load_dataframe(os.path.join(dataset_folder, 'movies.csv'))
raw_links_df = load_dataframe(os.path.join(dataset_folder, 'links.csv'))
raw_ratings_df = load_dataframe(os.path.join(dataset_folder, 'ratings.csv'))
raw_tags_df = load_dataframe(os.path.join(dataset_folder, 'tags.csv'))
raw_genome_tags_df = load_dataframe(os.path.join(dataset_folder, 'genome-tags.csv'))
raw_genome_scores_df = load_dataframe(os.path.join(dataset_folder, 'genome-scores.csv'))

In [17]:
raw_movies_df.show(10, truncate=False, vertical=False) # | movieId | title  | genres |
# TODO: Release date of movies are in `title` column, lets take it out later
raw_links_df.show(10, truncate=False, vertical=False) # | movieId | imdbId | tmdbId |
raw_ratings_df.show(10, truncate=False, vertical=False) #| userId | movieId | rating | timestamp |
raw_tags_df.show(10, truncate=False, vertical=False) # | userId | movieId | tag | timestamp |
raw_genome_tags_df.show(10, truncate=False, vertical=False) # | tagId | tag |
raw_genome_scores_df.show(10, truncate=False, vertical=False) # | movieId | tagId | relevance |

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



Computing `released_year` column and adding to `raw_movies_dataset`

In [122]:
raw_movies_df = raw_movies_df.withColumn(
    'release_year', 
    regexp_extract('title', r'\(([^)]*)\)', 1)
)



In [118]:
raw_movies_df.show(10, truncate=False, vertical=False) 

+-------+----------------------------------+-------------------------------------------+----+
|movieId|title                             |genres                                     |year|
+-------+----------------------------------+-------------------------------------------+----+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |1995|
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II (1995)|Comedy                                     |1995|
|6      |Heat (1995)                       |Action|Crime|Thriller                      |1995|
|7      |Sabrina (1995)                    |Comedy|Romance                             |1995|
|8      |Tom and Huck (1995)               |Adventure|Childr